<a href="https://colab.research.google.com/github/JohannRannou/DimensionnementVelo/blob/master/CLT_a_completer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mise en œuvre du module de CLT

Il y a  ici un certain nombre de fonctions destinées à faire les calculs élémentaires à réaliser les différentes opérations nécessaires traiter la CLT.

Ces fonctions de ne sont pas terminée, elles comportent des "trous" matérialiés par des ...

À vous de compléter à partir de votre compréhension du cours.





Plus bas dans le notebook il y a la section "Test du module". Elle va vous permettre de valider votre implémentation à partir d'une solution de référence. Pour cela, il vous faut activer le flag de debug :

    DEBUG = True

Quand il sera validé, pour pourrez le désactivé pour éviter des affichages inutiles :

    DEBUG = False

In [ ]:
DEBUG = True


## Initialisation diverses

In [ ]:
from numpy import *
from numpy.linalg import *
from matplotlib.pyplot import *
from matplotlib.patches import Rectangle

In [ ]:
#Ne pas changer la valeur de cette variable
DEBUG_DICT = {}

#Pour formatter l'affichage des matrices
set_printoptions(suppress=True,formatter={'float': '{: 0.2e}'.format})


## Calcul de la matrice ABBD

In [ ]:
def compute_ABBD(prop, lay_up):
    """Calcule la matrice ABBD

    Arguments:
        prop (dict): propriétés matériaux
        lay_up (list[int]): liste des angles du stratifié
    """

    #On boucle sur les plis pour calculer chaque terme de la matrice ABBD
    A=zeros((3,3))
    B=zeros((3,3))
    D=zeros((3,3))

    h=prop['thickness']
    
    # Dans la boucle ci-dessous, i est l'indice du pli (commence à 0) et angle
    # sera l'angle 
    for k, angle in enumerate(lay_up):
        # Calcul des hauteurs  h et h+1 du pli courant
        # la fonction compute_ply_heights est déjà remplie pour vous, mais
        # ça ne vous empêche pas d'y jeter un coup d'oeil ...
        h_k, h_kp1=compute_ply_heights(k, prop['thickness'], len(lay_up))

        # Calcule la matrice de raideur du pli courant dans le repère global
        Qg = computeGlobalQ(prop, angle) 

        # Calcul de A, composante par composante
        for i in range(3):
            for j in range(3):
                A[i,j] = A[i,j] + Qg[i,j]*(...)

        # Calcul de B, composante par composante
        for i in range(3):
            for j in range(3):
                B[i,j] = B[i,j] + Qg[i,j]*(...))

        # Calcul de D, composante par composante
        for i in range(3):
            for j in range(3):
                D[i,j] = D[i,j]+ Qg[i,j]*(...)

    #Assemblage de la matrice ABBD à partir des matrices élémentaires A,B et D
    ABBD=vstack( (hstack((A,B)),hstack((B,D))) )

    # Subsidiaire : la matrice ABBD telle qu'elle serait obtenue avec la
    # convention de Voigt
    ABBDVoigt=ABBD.copy()
    ABBDVoigt[ix_([2,5],[0,1,3,4])]*=1./sqrt(2)
    ABBDVoigt[ix_([2,5],[2,5])]*=1./2.
    ABBDVoigt[ix_([0,1,3,4],[2,5])]*=1./sqrt(2)

    if DEBUG:
        global DEBUG_DICT
        s = 'ABBD'
        if s not in DEBUG_DICT.keys():
            print('DEBUG dans compute_ABBD()')
            print(s,'=\n',ABBD)
            print()
            DEBUG_DICT[s]='printed'
        
    return ABBD

## Calcul automatique de $h_i$ et $h_{i+1}$ 

In [ ]:
def compute_ply_heights(i, thickness, nb_plies):
    """Calcule la hauteur inférieure et supérieure du pli.
       on a besoin du nombre total de pli pour savoir positionner chaque pli
       par rapport à une ligne moyenne
    Notes:
        Il n'y a rien à compléter ici
    
    """

    #décalage par rapport à la ligne moyenne
    h = -nb_plies*thickness/2.

    h = h + i*thickness
    hp1 = h + thickness

    return h, hp1

## Calcul de $Q$ dans la base matériaux

In [ ]:
def computeMaterialQ(prop):
    """Calcule et retourne la matrice de rigidité dans le repère matériau
    """
    #On commence par calculer S puis on l'inversera. C'est plus simple

    El = prop['El']
    Et = prop['Et']
    nult = prop['nult']
    Glt = prop['Glt']


    S=zeros((3,3))
    S[0,0] = ...
    S[0,1] = ...
    ...

    Qm=inv(S)
    
    if DEBUG:
        global DEBUG_DICT
        s = 'Qm'
        if s not in DEBUG_DICT.keys():
            print('DEBUG dans computeMaterialQ()')
            print(s,'=\n',Qm)
            print()
            DEBUG_DICT[s]='printed'

    return Qm

## Calcul de la matrice de rotation

Rappel :  Les matrices de rotation $P$ et son inverse $P^{-1}$ sont définies comme dans le cours et sont telles que :

$$Q^m=P^{-1}.Q^g.P$$
    

In [ ]:
def computeRotationMatrix(angle):
    """ Retourne la matrice de rotation en convention de Mandel 
    et en contraintes plannes

    Note:
        angle en degrés

    Les matrices de rotation P et son inverse Pi sont définies comme
    dans le cours et sont telles que
    :math:`Q^m=Pi.Q^g.P`
    """

    c=cos(angle*pi/180.)
    s=sin(angle*pi/180.)

    Pi=zeros((3,3))

    Pi[0,0] = ...
    Pi[0,1] = ...
    Pi[0,2] = ...
    ...

    P=inv(Pi)

    if DEBUG:
        global DEBUG_DICT
        s = 'P pour angle {}'.format(angle)
        if s not in DEBUG_DICT.keys():
            print('DEBUG dans computeRotationMatrix()')
            print(s,'=\n',P)
            print()
            DEBUG_DICT[s]='printed'

    return P,Pi

## Calcul de $Q$ dans le repère global

In [ ]:


def computeGlobalQ(prop, angle):
    """Retourne la matrice de raideur dans le repère global

    On rappelle que pour faire un produit de matrice AB, il faut 
    faire ``A @ B``. Du coup, pour faire un produit de 
    3 matrices ABC, il faut faire ``dot(A, dot(B, C))``

    """
    #Calcule la matrice de raideur dans le repère matériau
    Qm = computeMaterialQ(prop)
    P, Pi = computeRotationMatrix(angle)

    Qg = ...

    if DEBUG:
        global DEBUG_DICT
        s = 'Qg pour angle {}'.format(angle)
        if s not in DEBUG_DICT.keys():
            print('DEBUG dans computeGlobalQ()')
            print(s,'=\n',Qg)
            print()
            DEBUG_DICT[s]='printed'


    return Qg


## Calcul des déformations $\varepsilon_0$ et $\kappa$

In [ ]:
def compute_macroscopic_strains(N, M, ABBD):
    """Calcule les déformations macro :math:`\epsilon_0` et :math:`\kappa`

    La charge appliquée est composée des 2 vecteurs :

    Args:
        N(vecteur de taille 3): les forces généralisées en convention de Mandel
        M(vecteur de taille 3): les moments généralisés en convention de Mandel

    Returns:
        2 vecteurs de taille 3 ( :math:`\epsilon_0` et :math:`\kappa` ) en 
        convention de Mandel
    """

    NM=hstack((N,M))

    epsilon= inv(ABBD) ... NM

    epsilon0=epsilon[0:3]
    kappa=epsilon[3:6]
        
    if DEBUG:
        global DEBUG_DICT
        s = 'epsilon0, kappa'
        if s not in DEBUG_DICT.keys():
            print('DEBUG dans compute_macroscopic_strains()')
            print(s,'=\n',epsilon0,'\n', kappa)
            print()
            DEBUG_DICT[s]='printed'

    return epsilon0, kappa
    

## Calcul de $\varepsilon$ dans le repère global

In [ ]:
def computeEpsilonGlobal(epsilon0, kappa, z):
    """Retourne les déformations dans le repère global à une abscisse z. 

    Convention de Mandel -> vecteur de taille 3 : e11, e22, sqrt(2)e12.
    """

    epsilon_global = ...
    
    return epsilon_global

## Calcul de $\varepsilon$ dans le repère matériau


In [ ]:
def computeEpsilonMaterial(angle, epsilon0, kappa, z):
    """Retourne les déformations dans le repère local à une abscisse z.

    Convention de Mandel -> vecteur de taille 3 : e11, e22, sqrt(2)e12.

    On se sert de la matrice de rotation du pli ou de son inverse (telle que 
    défini dans le cours) pour la calculer
    """

    P, Pi = computeRotationMatrix(angle)
    epsilonGlobal = computeEpsilonGlobal(epsilon0, kappa, z)

    # Rappel de syntaxe : multiplication matrice vecteur : C = dot(A,B)    
    epsilonMaterial = ...

    return epsilonMaterial

## Calcul de $\sigma$ dans le repère matériau

In [ ]:
def computeSigmaMaterial(epsilon0, kappa, z, prop, angle):
    """Retourne les contraintes dans le repère local à une abscisse z.

    vecteur de taille 3 : s11, s22, sqrt(2)s12)
    """

    epsilonMaterial=computeEpsilonMaterial(angle, epsilon0, kappa, z)
    Qm = computeMaterialQ(prop)

    sigmaMaterial = ...

    return sigmaMaterial

## Calcul du critère de rupture de Wu

In [ ]:
def computeFailureCriterion(epsilon0, kappa, z, prop, angle):
    """Retourne le critère de rupture de Tsai-Wu  à une abscisse z. 
    """

    sigmaMaterial=computeSigmaMaterial(epsilon0, kappa, z, prop, angle)

    s11=sigmaMaterial[0]
    s22=sigmaMaterial[1]
    s12=sigmaMaterial[2]/sqrt(2)

    #Critère de Tsai-Wu
    Xt=prop['Xt']
    Xc=prop['Xc']
    Yt=prop['Yt']
    Yc=prop['Yc']
    Sc=prop['Sc']

    f1 = ...
    f2 = ...
    f6 = ...
    F11 = ...
    F22 = ...
    F12 = ...
    F66 = ...
    TsaiWu = ...

    return TsaiWu

## Fonctions de tracé

**Vous n'avez pas à y intervenir**

In [ ]:
def plotEpsilonMacro(epsilon0, kappa, prop, lay_up):
    """
    Réalise un bel affichage des déformations Macro (les trois composantes dans
    le repère global) dans l'épaisseur du stratifié
    """
    figure('epsilon macro')
    title('epsilon macro')

    #Pour chaque pli on rajoute une ligne
    for i, angle in enumerate(lay_up):

        # Calcul des hauteurs  h et h+1 du le pli courant
        # C'est comme dans le calcul de ABBD
        h_k, h_kp1=compute_ply_heights(i, prop['thickness'], len(lay_up))

        epsilon_h_k=computeEpsilonGlobal(epsilon0, kappa, h_k)
        epsilon_h_kp1=computeEpsilonGlobal(epsilon0, kappa, h_kp1)

        plot([epsilon_h_k[0],epsilon_h_kp1[0]],[h_k,h_kp1],'-o',lw=4,
             ms=10,alpha=0.8,label=r'$\varepsilon_{xx}$',color='r')
        plot([epsilon_h_k[1],epsilon_h_kp1[1]],[h_k,h_kp1],'-s',lw=4,
             ms=10,alpha=0.8,label=r'$\varepsilon_{yy}$',color='b')
        plot([epsilon_h_k[2]/sqrt(2),epsilon_h_kp1[2]/sqrt(2)],[h_k,h_kp1],
             '-<',lw=4,ms=10,alpha=0.8,label=r'$\varepsilon_{xy}$',color='y')
        
    xlabel(r'$\varepsilon$ dans le repere global',fontsize=14)

    addDecorationToPlot(lay_up, prop)
    # savefig('eg.pdf')


def plotEpsilonPerPly(epsilon0, kappa, prop, lay_up):
    """            
    Réalise un bel affichage des déformations dans chaque pli dans le repère 
    matériau (les trois composantes) dans l'épaisseur du stratifié
    """
    figure('epsilon (base materiau)')
    title('epsilon (base materiau)')

    for i, angle in enumerate(lay_up):
        #Calcul des hauteurs h et h+1 du le pli courant 
        h_k, h_kp1 = compute_ply_heights(i, prop['thickness'], len(lay_up))

        epsilon_h_k_m=computeEpsilonMaterial(angle, epsilon0, kappa, h_k)
        epsilon_h_kp1_m=computeEpsilonMaterial(angle, epsilon0, kappa, h_kp1)


        plot([epsilon_h_k_m[0],epsilon_h_kp1_m[0]],[h_k,h_kp1],'-o',lw=4,ms=10,
             alpha=0.8,label=r'$\varepsilon_{11}$',color='r')
        plot([epsilon_h_k_m[1],epsilon_h_kp1_m[1]],[h_k,h_kp1],'-s',lw=4,ms=10,
             alpha=0.8,label=r'$\varepsilon_{22}$',color='b')
        plot([epsilon_h_k_m[2]/sqrt(2),epsilon_h_kp1_m[2]/sqrt(2)],[h_k,h_kp1],
             '-<',lw=4,ms=10,alpha=0.8,label=r'$\varepsilon_{12}$',color='y')

    addDecorationToPlot(lay_up, prop)
    xlabel(r'$\varepsilon$ dans le repere materiau',fontsize=14)
    # savefig('em.pdf')




def plotStressPerPly(epsilon0, kappa, prop, lay_up):
    """
    Réalise un bel affichage des contraintes dans chaque pli dans le 
    repère matériau (les trois composantes) dans l'épaisseur du stratifié
    """
    figure('contrainte (base materiau)')
    title('contrainte (base materiau)')

    for i, angle in enumerate(lay_up):
        #Calcul des hauteurs h et h+1 du le pli courant 
        h_k, h_kp1 = compute_ply_heights(i, prop['thickness'], len(lay_up))

        sigma_h_k_m = computeSigmaMaterial(epsilon0, kappa, h_k, prop, angle)
        sigma_h_kp1_m = computeSigmaMaterial(epsilon0, kappa, h_kp1, prop, angle)


        plot([sigma_h_k_m[0], sigma_h_kp1_m[0]], [h_k,h_kp1], '-o', lw=4, ms=10,
             alpha=0.8,label=r'$\sigma_{11}$',color='r')
        plot([sigma_h_k_m[1],sigma_h_kp1_m[1]],[h_k,h_kp1],'-s',lw=4,ms=10,
             alpha=0.8,label=r'$\sigma_{22}$',color='b')
        plot([sigma_h_k_m[2]/sqrt(2),sigma_h_kp1_m[2]/sqrt(2)],[h_k,h_kp1],
             '-<',lw=4,ms=10,alpha=0.8,label=r'$\sigma_{12}$',color='y')

    addDecorationToPlot(lay_up, prop)
    xlabel(r'$\sigma$ dans le repere materiau',fontsize=14)
    # savefig('sm.pdf')



def plotFailureCriterion(epsilon0, kappa, prop, lay_up):
    """
    Réalise un bel affichage du critère de rupture  dans chaque pli.
    """
    figure('critere de rupture')
    title('critere de rupture')


    for i, angle in enumerate(lay_up):
        #Calcul des hauteurs h et h+1 du le pli courant 
        h_k, h_kp1 = compute_ply_heights(i, prop['thickness'], len(lay_up))

        zArray=linspace(h_k, h_kp1, 25)

        valeursCritere=empty_like(zArray)
        for i,z in enumerate(zArray):
            valeursCritere[i] = computeFailureCriterion(epsilon0, kappa, z,
                                                        prop, angle)

        plot(valeursCritere,zArray,'-',lw=4,ms=10,alpha=0.8,
             label=u'critère de ruture',color='r')

    addDecorationToPlot(lay_up, prop)
    addFailureCriterionDecorationToPlot(lay_up, prop)
    xlabel(u'critère de rupture',fontsize=14)
    # savefig('cr.pdf')


def addDecorationToPlot(lay_up, prop):
    """
    Ajoute les décorations aux graphiques 
    """
    #Donne les bornes actuelles du graphique
    x0,x1,y0,y1 = axis()

    currentAxis = gca()

    #Pour limiter le nombre de légendes aux 3 premières
    handles, labels = currentAxis.get_legend_handles_labels()
    legend(handles[:3], labels[:3],loc='lower right')


    for i, angle in enumerate(lay_up):
        #Calcul des hauteurs  h et h+1 du le pli courant 
        h_k, h_kp1=compute_ply_heights(i, prop['thickness'], len(lay_up))

        currentAxis.add_patch(Rectangle((x0,h_k), x1-x0, prop['thickness'],
                                        alpha=0.3,facecolor='#BBBBBB',
                                        edgecolor='k', linewidth=2))

        text(x0+(x1-x0)*0.025, h_k+prop['thickness']/2., 'Pli '+str(i+1)+' : '+
             str(angle)+u'°', ha='left')


def addFailureCriterionDecorationToPlot(lay_up, prop):
    """
    Ajoute une zone rouge quand le critère est atteint
    """
    #Donne les bornes actuelles du graphique
    x0,x1,y0,y1=axis()

    currentAxis = gca()

    #On ne trace pas la legende
    legend().set_visible(False)

    #On rajoute les rectangles rouges

    for i, angle in enumerate(lay_up):
        #Calcul des hauteurs h et h+1 du le pli courant 
        h_k, h_kp1 = compute_ply_heights(i, prop['thickness'], len(lay_up))

        currentAxis.add_patch(Rectangle((1.,h_k), x1-1., prop['thickness'],
                                        alpha=0.3, facecolor='#BB0000'))


def polar_negative(x, y, **kwargs):
    """
    Depuis la version 1.2.1 de matplotlib, polar ne trace plus les valeurs 
    négatives.
    Ici on trace les valeurs négatives, mais en pointillé.
    """

    # ax = kwargs.pop('ax', gca())
    # ls = kwargs.pop('ls')
    neg_filter = y<0
    pos_filter = np.logical_not(neg_filter)
    base_line, = polar(x[pos_filter], y[pos_filter], **kwargs)
    # polar(x+pi/180., -y, ls='--', color=base_line.get_color())
    polar(x[neg_filter], -y[neg_filter], ls='--', color=base_line.get_color())




def trace_diagramme_polaire(prop, lay_up, inverse=False):

    """
    Trace les diagrammes polaires de la matrice ABBD

    inverse = True : on trace alors les composantes de souplesse au lieu de raideur
    """

    global DEBUG
    store_DEBUG = DEBUG


    DEBUG = False
    
    # On fait tourner le stratifié entre 0° et 360° pour tracer les diagrammes polaires
    alpha=linspace(0.,360.,361)
    ABBD=empty((361,6,6))
    for i, a in enumerate(alpha):
        lay_up_a = [x-a for x in lay_up]
        ABBD_i = compute_ABBD(prop, lay_up_a)
        ABBD[i,:,:] = ABBD_i
        dot = ''
        if inverse:
            ABBD[i,:,:]=inv(ABBD[i,:,:])
            dot = '\overline'
            
  
    figure('Matrice A')

    polar_negative(alpha*pi/180.,ABBD[:,0,0], ls='-',label='$'+dot+'{A}_{11}$',lw=2.,alpha=1., zorder=-1)
    polar_negative(alpha*pi/180.,ABBD[:,0,1], ls='-',label='$'+dot+'{A}_{12}$',lw=2.,alpha=1., zorder=-2)
    polar_negative(alpha*pi/180.,ABBD[:,0,2], ls='-',label='$'+dot+'{A}_{16}$',lw=2.,alpha=1., zorder=-3)
    polar_negative(alpha*pi/180.,ABBD[:,1,1], ls='-',label='$'+dot+'{A}_{22}$',lw=2.,alpha=1., zorder=-4)
    polar_negative(alpha*pi/180.,ABBD[:,1,2], ls='-',label='$'+dot+'{A}_{26}$',lw=2.,alpha=1., zorder=-5)
    polar_negative(alpha*pi/180.,ABBD[:,2,2], ls='-',label='$'+dot+'{A}_{66}$',lw=2.,alpha=1., zorder=-6)
    legend(loc='upper left')
    # savefig('A.pdf')

    figure('Matrice B')

    polar_negative(alpha*pi/180.,ABBD[:,3,0], ls='-',label='$'+dot+'{B}_{11}$',lw=2.,alpha=1., zorder=-1)
    polar_negative(alpha*pi/180.,ABBD[:,3,1], ls='-',label='$'+dot+'{B}_{12}$',lw=2.,alpha=1., zorder=-2)
    polar_negative(alpha*pi/180.,ABBD[:,3,2], ls='-',label='$'+dot+'{B}_{16}$',lw=2.,alpha=1., zorder=-3)
    polar_negative(alpha*pi/180.,ABBD[:,4,1], ls='-',label='$'+dot+'{B}_{22}$',lw=2.,alpha=1., zorder=-4)
    polar_negative(alpha*pi/180.,ABBD[:,4,2], ls='-',label='$'+dot+'{B}_{26}$',lw=2.,alpha=1., zorder=-5)
    polar_negative(alpha*pi/180.,ABBD[:,5,2], ls='-',label='$'+dot+'{B}_{66}$',lw=2.,alpha=1., zorder=-6)
    legend(loc='upper left')
   # # savefig('B.pdf')

    figure('Matrice D')

    polar_negative(alpha*pi/180.,ABBD[:,3,3], ls='-',label='$'+dot+'{D}_{11}$',lw=2.,alpha=1., zorder=-1)
    polar_negative(alpha*pi/180.,ABBD[:,3,4], ls='-',label='$'+dot+'{D}_{12}$',lw=2.,alpha=1., zorder=-2)
    polar_negative(alpha*pi/180.,ABBD[:,3,5], ls='-',label='$'+dot+'{D}_{16}$',lw=2.,alpha=1., zorder=-3)
    polar_negative(alpha*pi/180.,ABBD[:,4,4], ls='-',label='$'+dot+'{D}_{22}$',lw=2.,alpha=1., zorder=-4)
    polar_negative(alpha*pi/180.,ABBD[:,4,5], ls='-',label='$'+dot+'{D}_{26}$',lw=2.,alpha=1., zorder=-5)
    polar_negative(alpha*pi/180.,ABBD[:,5,5], ls='-',label='$'+dot+'{D}_{66}$',lw=2.,alpha=1., zorder=-6)
    legend(loc='upper left')
   # # savefig('D.pdf')

    DEBUG = store_DEBUG

def trace_eps_sig_crit(epsilon0, kappa, prop_pli, lay_up):
    """Trace les déformation, contrainte et critère à rupture

    * Trace les déformations dans le repère global et le repère matériau propre au pli
    * Trace les contraintes dans le repère du pli
    * Trace le critère de rupture
    """

    
    #On trace les déformations macroscopique dans l'épaisseur du stratifié
    plotEpsilonMacro(epsilon0, kappa, prop_pli, lay_up)

    # On trace les déformations dans chaque pli (dans le repère matériau)
    # dans l'épaisseur du stratifié
    plotEpsilonPerPly(epsilon0, kappa, prop_pli, lay_up)

    # #On trace les contraintes dans chaque pli (dans le repère matériau)
    # dans l'épaisseur du stratifié
    plotStressPerPly(epsilon0, kappa, prop_pli, lay_up)

    # #On trace le critère de rupture de Tsai-Wu
    plotFailureCriterion(epsilon0, kappa, prop_pli, lay_up)


def trace_crit(epsilon0, kappa, prop_pli, lay_up):
    """Trace le critère de rupture uniquemet
    """
    # #On trace le critère de rupture de Tsai-Wu
    plotFailureCriterion(epsilon0, kappa, prop_pli, lay_up)


 # Test du module

In [ ]:
#On définit les propriétés du pli dans un dictionnaire
if DEBUG:
  prop_pli = {}
  prop_pli['El'] = 126000.
  prop_pli['Et'] = 11000.
  prop_pli['Glt'] = 6600.
  prop_pli['nult'] = 0.28
  prop_pli['thickness'] = 0.25
  #On complète avec les propriétés de rupture
  prop_pli['Xt'] = 1950.
  prop_pli['Xc'] = -1000.
  prop_pli['Yt'] = 48.
  prop_pli['Yc'] = -200.
  prop_pli['Sc'] = 79.

  #On définit la stratification dans une liste
  lay_up_A=[0., 90.,  45., 0.]

Après avoir éxécuté la cellule suivante, avec DEBUG=True, vous devez obtenir la sortie de référence suivante :
```
DEBUG dans computeMaterialQ()
Qm =
 [[ 1.27e+05  3.10e+03  0.00e+00]
 [ 3.10e+03  1.11e+04  0.00e+00]
 [ 0.00e+00  0.00e+00  1.32e+04]]

DEBUG dans computeRotationMatrix()
P pour angle 0.0 =
 [[ 1.00e+00  0.00e+00  0.00e+00]
 [ 0.00e+00  1.00e+00  0.00e+00]
 [ 0.00e+00  0.00e+00  1.00e+00]]

DEBUG dans computeGlobalQ()
Qg pour angle 0.0 =
 [[ 1.27e+05  3.10e+03  0.00e+00]
 [ 3.10e+03  1.11e+04  0.00e+00]
 [ 0.00e+00  0.00e+00  1.32e+04]]

[[ 1.27e+05  3.10e+03  0.00e+00]
 [ 3.10e+03  1.11e+04  0.00e+00]
 [ 0.00e+00  0.00e+00  1.32e+04]]
DEBUG dans computeRotationMatrix()
P pour angle 90.0 =
 [[ 3.75e-33  1.00e+00 -8.66e-17]
 [ 1.00e+00  3.75e-33  8.66e-17]
 [ 8.66e-17 -8.66e-17 -1.00e+00]]

DEBUG dans computeGlobalQ()
Qg pour angle 90.0 =
 [[ 1.11e+04  3.10e+03  4.52e-13]
 [ 3.10e+03  1.27e+05  9.57e-12]
 [ 4.52e-13  9.57e-12  1.32e+04]]

[[ 1.27e+05  3.10e+03  0.00e+00]
 [ 3.10e+03  1.11e+04  0.00e+00]
 [ 0.00e+00  0.00e+00  1.32e+04]]
DEBUG dans computeRotationMatrix()
P pour angle 45.0 =
 [[ 5.00e-01  5.00e-01 -7.07e-01]
 [ 5.00e-01  5.00e-01  7.07e-01]
 [ 7.07e-01 -7.07e-01  1.57e-16]]

DEBUG dans computeGlobalQ()
Qg pour angle 45.0 =
 [[ 4.26e+04  2.94e+04  4.09e+04]
 [ 2.94e+04  4.26e+04  4.09e+04]
 [ 4.09e+04  4.09e+04  6.59e+04]]

[[ 1.27e+05  3.10e+03  0.00e+00]
 [ 3.10e+03  1.11e+04  0.00e+00]
 [ 0.00e+00  0.00e+00  1.32e+04]]
DEBUG dans compute_ABBD()
ABBD =
 [[ 7.69e+04  9.69e+03  1.02e+04  9.86e+02  8.23e+02  1.28e+03]
 [ 9.69e+03  4.79e+04  1.02e+04  8.23e+02 -2.63e+03  1.28e+03]
 [ 1.02e+04  1.02e+04  2.64e+04  1.28e+03  1.28e+03  1.65e+03]
 [ 9.86e+02  8.23e+02  1.28e+03  9.53e+03  3.96e+02  2.13e+02]
 [ 8.23e+02 -2.63e+03  1.28e+03  3.96e+02  1.69e+03  2.13e+02]
 [ 1.28e+03  1.28e+03  1.65e+03  2.13e+02  2.13e+02  1.37e+03]]

matrice ABBD
[[ 7.69e+04  9.69e+03  1.02e+04  9.86e+02  8.23e+02  1.28e+03]
 [ 9.69e+03  4.79e+04  1.02e+04  8.23e+02 -2.63e+03  1.28e+03]
 [ 1.02e+04  1.02e+04  2.64e+04  1.28e+03  1.28e+03  1.65e+03]
 [ 9.86e+02  8.23e+02  1.28e+03  9.53e+03  3.96e+02  2.13e+02]
 [ 8.23e+02 -2.63e+03  1.28e+03  3.96e+02  1.69e+03  2.13e+02]
 [ 1.28e+03  1.28e+03  1.65e+03  2.13e+02  2.13e+02  1.37e+03]]

```




In [ ]:
if DEBUG:
  ABBD = compute_ABBD(prop_pli, lay_up_A)

  # On affiche la matrice ABBD qui a été calculée lors de la création du
  # stratifié (ligne précédente)
  print('matrice ABBD')
  print(ABBD)
  print()

In [ ]:
if DEBUG:
  #On définit un chargement via deux vecteurs N et M
  #Et on applique ce chargement
  N1=array([1500.,0.,0.])
  M1=array([0.,0.,0.])

  epsilon0, kappa = compute_macroscopic_strains(N1, M1, ABBD)
  print('epsilon0, kappa')
  print(epsilon0)
  print(kappa)

  trace_eps_sig_crit(epsilon0, kappa, prop_pli, lay_up_A)

In [ ]:
if DEBUG:
  trace_diagramme_polaire(prop_pli, lay_up_A)